In [1]:
import imageParse as imgP
import imageSent as imgS
import numpy as np
import pandas as pd
import tensorflow as tf

print(tf.__version__)

2.7.0


In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
tf.data.experimental.enable_debug_mode()

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
abs_X0, abs_Y0, abs_X1, abs_Y1 = imgP.parseData_Abs()
print(abs_X0.shape)
print(abs_X1.shape)

(224, 255, 255, 3)
(56, 255, 255, 3)


In [6]:
abs_X0 = imgP.input_prep_fn(abs_X0)
abs_X1 = imgP.input_prep_fn(abs_X1)
print(abs_X0.shape)
print(abs_X1.shape)

(224, 32, 32, 3)
(56, 32, 32, 3)


2022-12-10 15:50:43.310037: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
art_X0, art_Y0, art_X1, art_Y1 = imgP.parseData_Art()
print(art_X0.shape)
print(art_X1.shape)

(644, 255, 255, 3)
(161, 255, 255, 3)


In [8]:
art_X0 = imgP.input_prep_fn(art_X0)
art_X1 = imgP.input_prep_fn(art_X1)
print(art_X0.shape)
print(art_X1.shape)

(644, 32, 32, 3)
(161, 32, 32, 3)


In [9]:
VAD_map = np.array([
    [1,0.735,0.772], #0: 'happy'
    [0.918,0.61,0.566], #1: 'funny'
    [0.225,0.333,0.149], #2: 'sad'
    [0.63,0.52,0.509], #3: 'tender'
    [0.95,0.792,0.789], #4: 'exciting'
    [0.122,0.83,0.604], #5: 'angry'
    [0.062,0.952,0.528], #6: 'scary'
])

VAD_pd = pd.DataFrame(
    columns = ["valence", "arousal", "dominance"], 
    data = VAD_map, 
    index = ['happy', 'funny', 'sad', 'tender', 
             'exciting', 'angry', 'scary'])

In [10]:
abs_model = imgS.ImageSentModel(name='abs')
abs_model((abs_X0[:5], abs_Y0[:5]))
abs_model.summary()

Model: "abs"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  224       
                                                                 
 conv2d_1 (Conv2D)           multiple                  1168      
                                                                 
 conv2d_2 (Conv2D)           multiple                  4640      
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  491580    
                                                               

In [11]:
optimizer = tf.keras.optimizers.Adam(0.0001)
abs_model.compile(optimizer=optimizer, VAD_map=VAD_map)

In [12]:
abs_Y0_VAD = VAD_pd.loc[abs_Y0]
abs_Y1_VAD = VAD_pd.loc[abs_Y1]

In [13]:
abs_model.fit(
    (abs_X0, abs_Y0_VAD), abs_Y0_VAD,
    epochs     = 75,
    batch_size = 8,
);

Epoch 1/75
28/28 [==============================] - 1s 30ms/step - mse_loss: 0.6808 - acc: 0.1339
Epoch 2/75
28/28 [==============================] - 1s 31ms/step - mse_loss: 0.6440 - acc: 0.1339
Epoch 3/75
28/28 [==============================] - 1s 30ms/step - mse_loss: 0.6460 - acc: 0.1339
Epoch 4/75
28/28 [==============================] - 1s 28ms/step - mse_loss: 0.6236 - acc: 0.1339
Epoch 5/75
28/28 [==============================] - 1s 30ms/step - mse_loss: 0.6105 - acc: 0.1339
Epoch 6/75
28/28 [==============================] - 1s 30ms/step - mse_loss: 0.5913 - acc: 0.1339
Epoch 7/75
28/28 [==============================] - 1s 29ms/step - mse_loss: 0.5762 - acc: 0.1339
Epoch 8/75
28/28 [==============================] - 1s 28ms/step - mse_loss: 0.5468 - acc: 0.1339
Epoch 9/75
28/28 [==============================] - 1s 32ms/step - mse_loss: 0.5362 - acc: 0.1339
Epoch 10/75
28/28 [==============================] - 1s 28ms/step - mse_loss: 0.4991 - acc: 0.1339
Epoch 11/75
28/28 [

In [14]:
abs_model.evaluate(
    (abs_X1, abs_Y1_VAD), abs_Y1_VAD,
    batch_size = 8,
);

7/7 [==============================] - 0s 15ms/step - mse_loss: 1.2404 - acc: 0.1071


In [15]:
art_model = imgS.ImageSentModel(name='art')
art_model((art_X0[:5], art_Y0[:5]))
art_model.summary()

Model: "art"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           multiple                  224       
                                                                 
 conv2d_4 (Conv2D)           multiple                  1168      
                                                                 
 conv2d_5 (Conv2D)           multiple                  4640      
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  491580    
                                                               

In [16]:
art_model.compile(optimizer=optimizer, VAD_map=VAD_map)

In [17]:
art_Y0_VAD = VAD_pd.loc[art_Y0]
art_Y1_VAD = VAD_pd.loc[art_Y1]

In [18]:
art_model.fit(
    (art_X0, art_Y0_VAD), art_Y0_VAD,
    epochs     = 75,
    batch_size = 8,
);

Epoch 1/75
81/81 [==============================] - 3s 32ms/step - mse_loss: 0.7536 - acc: 0.1265
Epoch 2/75
81/81 [==============================] - 3s 36ms/step - mse_loss: 0.7058 - acc: 0.1265
Epoch 3/75
81/81 [==============================] - 3s 31ms/step - mse_loss: 0.6642 - acc: 0.1296
Epoch 4/75
81/81 [==============================] - 2s 30ms/step - mse_loss: 0.6157 - acc: 0.1281
Epoch 5/75
81/81 [==============================] - 3s 33ms/step - mse_loss: 0.5562 - acc: 0.1265
Epoch 6/75
81/81 [==============================] - 2s 30ms/step - mse_loss: 0.4929 - acc: 0.1281
Epoch 7/75
81/81 [==============================] - 3s 33ms/step - mse_loss: 0.4465 - acc: 0.1296
Epoch 8/75
81/81 [==============================] - 3s 32ms/step - mse_loss: 0.3866 - acc: 0.1281
Epoch 9/75
81/81 [==============================] - 3s 31ms/step - mse_loss: 0.3479 - acc: 0.1296
Epoch 10/75
81/81 [==============================] - 3s 33ms/step - mse_loss: 0.3053 - acc: 0.1296
Epoch 11/75
81/81 [

In [19]:
art_model.evaluate(
    (art_X1, art_Y1_VAD), art_Y1_VAD,
    batch_size = 8,
);

21/21 [==============================] - 0s 15ms/step - mse_loss: 1.0967 - acc: 0.1190
